<a href="https://colab.research.google.com/github/coordanalisis-sedatu/bateria_autoprod/blob/main/indica_autoprod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Borrar los datos del entorno
rm(list=ls())

In [ ]:
# Librerías
if (!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, janitor, openxlsx)

Loading required package: pacman

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘pacman’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘minqa’, ‘numDeriv’, ‘mitools’, ‘survey’



srvyr installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘snakecase’



janitor installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


openxlsx installed



In [ ]:
# Ver el timeout predeterminado
getOption('timeout')

[1] 60

In [ ]:
# Cambiar el timeout predeterminado
options(timeout=10000)


In [ ]:
# Función para descargar y descomprimir un archivo con timeout
download_and_unzip <- function(url, destination) {
  tryCatch({
    # Descargar el archivo con timeout
    download.file(url, destfile = paste0(destination, "/", basename(url)), mode = "wb", timeout = 10000)

    # Descomprimir el archivo
    unzip(paste0(destination, "/", basename(url)), exdir = destination)

    # Eliminar el archivo comprimido después de descomprimirlo
    unlink(paste0(destination, "/", basename(url)))

    message(paste("Descarga y descompresión exitosas para", basename(url)))
  }, error = function(e) {
    warning(paste("Error al descargar o descomprimir", basename(url), ":", conditionMessage(e)))
  })
}

In [ ]:
# URLs para descargar
urls <- c(
  "https://www.coneval.org.mx/Medicion/MP/Documents/MMP_2022/Programas_calculo/R_MMP_2022.zip",
  "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_viviendas_csv.zip",
  "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_concentradohogar_csv.zip"
)

# Carpeta de destino
destination <- "microdatos"

# Recorrer las URLs y descargar/descomprimir
for (url in urls) {
  download_and_unzip(url, destination)
}

Descarga y descompresión exitosas para R_MMP_2022.zip

Descarga y descompresión exitosas para enigh2022_ns_viviendas_csv.zip

Descarga y descompresión exitosas para enigh2022_ns_concentradohogar_csv.zip



In [ ]:
pobreza <- readr::read_csv("microdatos/Base final/pobreza_22.csv") %>%
  janitor::clean_names()

Rows: 309534 Columns: 79
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): folioviv
dbl (78): foliohog, numren, est_dis, upm, factor, tam_loc, rururb, ent, ubic...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
concentrado <- readr::read_csv("microdatos/concentradohogar.csv") %>%
  janitor::clean_names()

Rows: 90102 Columns: 126
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (5): folioviv, ubica_geo, est_dis, upm, educa_jefe
dbl (121): foliohog, tam_loc, est_socio, factor, clase_hog, sexo_jefe, edad_...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
# Carga y limpia el conjunto de datos de viviendas
viviendas <- read.csv("microdatos/viviendas.csv") %>%
  janitor::clean_names() %>%  #Poner las variables en minusculas y quitar las variables
  mutate(
    mat_pared = as.numeric(mat_pared),
    mat_pisos = as.numeric(mat_pisos),
    mat_techos = as.numeric(mat_techos),
    rezago = case_when(
      (tot_resid / num_cuarto) > 2.5 |   # Hacinamiento
        mat_pared %in% c(1, 2, 3, 4, 5, 6) |
        mat_pisos %in% c(1) |
        mat_techos %in% c(1, 2, 3, 4, 6, 7, 9) |
        excusado == 2 ~ "En rezago",
      TRUE ~ "Fuera de rezago"
    ),
    vivi_autoprod = case_when(
      (tenencia %in% c(3, 4) & tipo_adqui %in% c(2, 3)) ~ "Sí",
      TRUE ~ "No"
    ),
    hacinamiento = tot_resid / num_cuarto,
    hacinamiento = ifelse(hacinamiento > 2.5, 1, 0),
    folioviv = as.character(folioviv),
    # Clave de entidad federativa
    cve_ent = case_when(
      nchar(folioviv) == 10 ~ as.character(substr(folioviv, 1, 2)), #1 y 2 clave de Entidad Federativa
      nchar(folioviv) == 9 ~ as.character(paste0("0", substr(folioviv, 1, 1))),
      TRUE ~ as.character(folioviv)
    ),
    # Variable para obtener el total de residentes que se encuentran en viviendas autoproducidas.
      tot_resid_autoprod = case_when(
      vivi_autoprod == "Sí" ~ tot_resid,
      TRUE ~ 0
    ),
    # Variable para obtener el total de hogares que se encuentran en viviendas autoproducidas.
    tot_hog_autoprod = case_when(
      vivi_autoprod == "Sí" ~ tot_hog,
      TRUE ~ 0
    ),
    # Nombre de cada entidad
    nom_ent = case_when(
      cve_ent == "01" ~ "Aguascalientes",
      cve_ent == "02" ~ "Baja California",
      cve_ent == "03" ~ "Baja California Sur",
      cve_ent == "04" ~ "Campeche",
      cve_ent == "05" ~ "Coahuila de Zaragoza",
      cve_ent == "06" ~ "Colima",
      cve_ent == "07" ~ "Chiapas",
      cve_ent == "08" ~ "Chihuahua",
      cve_ent == "09" ~ "Ciudad de México",
      cve_ent == "10" ~ "Durango",
      cve_ent == "11" ~ "Guanajuato",
      cve_ent == "12" ~ "Guerrero",
      cve_ent == "13" ~ "Hidalgo",
      cve_ent == "14" ~ "Jalisco",
      cve_ent == "15" ~ "México",
      cve_ent == "16" ~ "Michoacán de Ocampo",
      cve_ent == "17" ~ "Morelos",
      cve_ent == "18" ~ "Nayarit",
      cve_ent == "19" ~ "Nuevo León",
      cve_ent == "20" ~ "Oaxaca",
      cve_ent == "21" ~ "Puebla",
      cve_ent == "22" ~ "Querétaro",
      cve_ent == "23" ~ "Quintana Roo",
      cve_ent == "24" ~ "San Luis Potosí",
      cve_ent == "25" ~ "Sinaloa",
      cve_ent == "26" ~ "Sonora",
      cve_ent == "27" ~ "Tabasco",
      cve_ent == "28" ~ "Tamaulipas",
      cve_ent == "29" ~ "Tlaxcala",
      cve_ent == "30" ~ "Veracruz de Ignacio de la Llave",
      cve_ent == "31" ~ "Yucatán",
      cve_ent == "32" ~ "Zacatecas",
      TRUE ~ "No identificado"
    )
  )

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `mat_pisos = as.numeric(mat_pisos)`.
Caused by warning:
! NAs introduced by coercion”


In [ ]:
#Num. de observaciones pobreza
obs_pobreza <- nrow(pobreza)
obs_pobreza

[1] 309534

In [ ]:
#Num. de observaciones concentrado
obs_concentrado <- nrow(concentrado)
obs_concentrado

[1] 90102

In [ ]:
#Num. de observaciones viviendas
obs_viviendas <- nrow(viviendas)
obs_viviendas

[1] 88823

In [ ]:
viviendas <- viviendas %>%
  # Realiza un join con el conjunto de datos 'concentrado'
  left_join(
    concentrado %>%
      group_by(folioviv) %>%
      filter(row_number() == 1) %>%
      select(folioviv, sexo_jefe, clase_hog, ing_cor),
    by = "folioviv"
  )

In [ ]:
viviendas <- viviendas %>%
  # Realiza un join con el conjunto de datos 'pobreza'
  left_join(
    pobreza %>%
      group_by(folioviv) %>%
      filter(row_number() == 1) %>%
      select(folioviv, pobreza, pobreza_e, ic_sbv),
    by = "folioviv"
  )

In [ ]:
viviendas<-viviendas%>%
  mutate(largo=nchar(cve_ent))

In [ ]:
summary(viviendas$cve_ent)

   Length     Class      Mode 
    88823 character character 

In [ ]:
unique(viviendas$nom_ent)

[1] "Aguascalientes"                  "Baja California"                
 [3] "Baja California Sur"             "Campeche"                       
 [5] "Coahuila de Zaragoza"            "Colima"                         
 [7] "Chiapas"                         "Chihuahua"                      
 [9] "Ciudad de México"                "Durango"                        
[11] "Guanajuato"                      "Guerrero"                       
[13] "Hidalgo"                         "Jalisco"                        
[15] "México"                          "Michoacán de Ocampo"            
[17] "Morelos"                         "Nayarit"                        
[19] "Nuevo León"                      "Oaxaca"                         
[21] "Puebla"                          "Querétaro"                      
[23] "Quintana Roo"                    "San Luis Potosí"                
[25] "Sinaloa"                         "Sonora"                         
[27] "Tabasco"                         "Tamaulipas"                     
[29] "Tlaxcala"                        "Veracruz de Ignacio de la Llave"
[31] "Yucatán"                         "Zacatecas"

In [ ]:
summarize_data <- function(data, grouping_vars, filter_expr = NULL) {

  data %>%
    filter(!!filter_expr) %>%
    group_by(!!!syms(grouping_vars)) %>%
    summarise(
      viviendas = survey_total(vartype = c("cv", "ci")),
      pct = survey_prop(vartype = c("cv", "ci"))
    ) %>%
    janitor::adorn_totals("row")
}

In [ ]:
#estructura general de la tabla
str(viviendas)

'data.frame':	88823 obs. of  78 variables:
 $ folioviv          : chr  "100005002" "100005003" "100005004" "100012002" ...
 $ tipo_viv          : chr  "1" "1" "1" "1" ...
 $ mat_pared         : num  8 8 8 8 8 8 8 8 8 8 ...
 $ mat_techos        : num  10 10 10 10 10 10 10 10 10 10 ...
 $ mat_pisos         : num  3 3 3 3 3 3 3 3 3 3 ...
 $ antiguedad        : int  30 15 25 50 45 25 48 36 35 1 ...
 $ antigua_ne        : int  NA NA NA NA NA NA NA NA NA NA ...
 $ cocina            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ cocina_dor        : int  2 2 2 2 2 2 2 2 2 2 ...
 $ cuart_dorm        : int  2 3 3 4 3 2 5 4 3 3 ...
 $ num_cuarto        : int  4 5 7 6 4 4 6 7 4 6 ...
 $ disp_agua         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ dotac_agua        : int  1 1 1 1 1 2 1 1 1 1 ...
 $ excusado          : int  1 1 1 1 1 1 1 1 1 1 ...
 $ uso_compar        : int  2 2 2 2 2 2 2 2 2 2 ...
 $ sanit_agua        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ biodigest         : int  2 2 2 2 2 2 2 2 2 2 ...
 $ bano_comp        

In [ ]:
#Opción para ajustar si se tiene una sola upm
options(survey.lmonely.psu="adjust")
#Definir diseño muestral
dm<-viviendas%>%
  #Diseño muestral
  as_survey_design(weights=factor, strata=est_dis, ids=upm, nest=TRUE)

In [ ]:
# ------------------------------------------------ Mostrar el total de viviendas dentro del diseño muestral
dm%>%
  summarise(viviendas=survey_total())

viviendas,viviendas_se
<dbl>,<dbl>
36968871,123818.8


In [ ]:
# ------------------------------------------------ Viviendas por tipo de adquisición del total dentro del diseño muestral
# Se tomará en cuenta las variables 2(La mandó construir) y 3(La construyó él mismo) para crear la variable de las viviendas autoproducidas.
dm %>%
    group_by(tipo_adqui) %>%
    summarise(
    viviendas = survey_total(vartype = c("cv"), level = 0.95)
    ) %>%
    mutate(
    tipo_adqui = case_when(
      tipo_adqui == 1 ~ "La compró hecha",
      tipo_adqui == 2 ~ "La mandó construir",
      tipo_adqui == 3 ~ "La construyó él mismo",
      tipo_adqui == 4 ~ "La obtuvo de otra manera",
      TRUE ~ "No especificado"
    )
  ) %>%
  janitor::adorn_totals()


tipo_adqui,viviendas,viviendas_cv
<chr>,<dbl>,<dbl>
La compró hecha,7927697,0.012951971
La mandó construir,9602694,0.011209743
La construyó él mismo,6543043,0.013907712
La obtuvo de otra manera,1639874,0.026130331
No especificado,11255563,0.008630989
Total,36968871,0.072830745


In [ ]:
# ------------------------------------------------  Viviendas por tipo de tenencia del total dentro del diseño muestral
# Se tomará en cuenta las variables 3 (Es propia pero la están pagando) y 4 (Es propia) para crear la variable de las viviendas autoproducidas.
dm %>%
    group_by(tenencia) %>%
    summarise(
    viviendas = survey_total(vartype = c("cv"), level = 0.95)
    ) %>%
    mutate(
    tenencia = case_when(
      tenencia == 1 ~ "Es rentada",
      tenencia == 2 ~ "Es prestada",
      tenencia == 3 ~ "Es propia pero la están pagando",
      tenencia == 4 ~ "Es propia",
      tenencia == 5 ~ "Está intestada o en litigio",
      tenencia == 6 ~ "Presenta carencia",
      TRUE ~ "No especificado"
    )
  ) %>%
  janitor::adorn_totals()

tenencia,viviendas,viviendas_cv
<chr>,<dbl>,<dbl>
Es rentada,5566173,0.01364258
Es prestada,4533666,0.01475087
Es propia pero la están pagando,2910497,0.02325305
Es propia,22802811,0.00609777
Está intestada o en litigio,876601,0.03419609
Presenta carencia,279123,0.05398445
Total,36968871,0.14592482


In [ ]:
# ------------------------------------------------ Viviendas Autoproducidas

# Dentro del filtro de las viviendas De tipo de tenencia 2 (Es propia pero la están pagando) y 3(Es propia),
# Se creará un filtro por el tipo de adquisición 3(La mandó construir) y 4(La construyó él mismo)
# para saber el total de viviendas autoproducidas.

dm %>%
  filter(tipo_adqui %in% c(2, 3)) %>%
  mutate(
  tenencia = case_when(
      tenencia == 3 ~ "Es propia pero la están pagando",
      tenencia == 4 ~ "Es propia")) %>%
  group_by(tenencia) %>%
  summarise(
    viviendas = survey_total(vartype = "cv"),
    porcentaje = survey_prop(vartype = "cv")
  ) %>%
  janitor::adorn_totals()



tenencia,viviendas,viviendas_cv,porcentaje,porcentaje_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Es propia,15969781,0.008393368,0.98910201,0.0006677747
Es propia pero la están pagando,175956,0.060802190,0.01089799,0.0606072843
Total,16145737,0.069195558,1.00000000,0.0612750589


In [ ]:
# ------------------------------------------------ Mostrar las viviendas que son Autoproducidas dentro del diseño muestral
dm %>%
  group_by(vivi_autoprod) %>%
  summarise(viviendas = survey_total(vartype = "cv", level = 0.95)) %>%
  janitor::adorn_totals()

vivi_autoprod,viviendas,viviendas_cv
<chr>,<dbl>,<dbl>
No,20823134,0.006196205
Sí,16145737,0.008336528
Total,36968871,0.014532733


In [ ]:
# ------------------------------------------------ Mostrar las viviendas que se encuentran en condición de rezago dentro del diseño muestral
dm %>%
  group_by(rezago) %>%
  summarise(viviendas = survey_total(vartype = "cv", level = 0.95)) %>%
  janitor::adorn_totals()

rezago,viviendas,viviendas_cv
<chr>,<dbl>,<dbl>
En rezago,8977409,0.011080097
Fuera de rezago,27991462,0.004748287
Total,36968871,0.015828384


In [ ]:
# ------------------------------------------------ Viviendas en condición de rezago que son Autoproducidas
dm %>%
  filter(vivi_autoprod == "Sí") %>%
  group_by(rezago) %>%
  summarise(viviendas = survey_total(vartype = "cv", level = 0.95)) %>%
  janitor::adorn_totals()


rezago,viviendas,viviendas_cv
<chr>,<dbl>,<dbl>
En rezago,5310725,0.01417899
Fuera de rezago,10835012,0.01111819
Total,16145737,0.02529718


In [ ]:
# ------------------------------------------------ Total de residentes que habitan en viviendas
dm %>%
  summarise(residentes = survey_total(tot_resid, vartype = "cv", level = 0.95)) %>%
  janitor::adorn_totals()

residentes,residentes_cv
<chr>,<dbl>
128999038,0.004676614
Total,0.004676614


In [ ]:
# ------------------------------------------------ Total de residentes que habitan viviendas autoproducidas
dm %>%
  summarise(residentes = survey_total(tot_resid_autoprod, vartype = "cv", level = 0.95)) %>%
  janitor::adorn_totals()

residentes,residentes_cv
<chr>,<dbl>
60426775,0.009749827
Total,0.009749827


In [ ]:
dm %>%
filter(vivi_autoprod == "Sí")%>%
summarise(residentes = survey_total(tot_resid, vartype = "cv")) %>%
  janitor::adorn_totals()


residentes,residentes_cv
<chr>,<dbl>
60426775,0.009749827
Total,0.009749827


In [ ]:
# ------------------------------------------------
# 1. Total de viviendas particulares habitadas autoproducidas
 dm %>%
  filter(vivi_autoprod == "Sí") %>%
  summarise(viviendas = survey_total(vartype = c("cv"))) %>%
  janitor::adorn_totals()



viviendas,viviendas_cv
<chr>,<dbl>
16145737,0.008336528
Total,0.008336528


In [ ]:
# ------------------------------------------------ 1. Total de viviendas particulares habitadas autoproducidas
tot_vivi <- dm %>%
  filter(vivi_autoprod == "Sí") %>%
  summarise(viviendas = survey_total(vartype = "cv")) %>%
  janitor::adorn_totals()

tot_vivi


viviendas,viviendas_cv
<chr>,<dbl>
16145737,0.008336528
Total,0.008336528


In [ ]:
# ------------------------------------------------ 2. Total de viviendas particulares habitadas autoproducidas por entidad federativa
tot_vivi_nom_ent <- dm %>%
  filter(vivi_autoprod=="Sí")%>%
  group_by(nom_ent) %>%
  summarise(
    viviendas = survey_total(vartype = "cv"),
    porcentaje = survey_prop(vartype = "cv")) %>%
  janitor::adorn_totals()

tot_vivi_nom_ent

nom_ent,viviendas,viviendas_cv,porcentaje,porcentaje_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Aguascalientes,109512,0.06191309,0.006782719,0.06205424
Baja California,320884,0.05600397,0.019874224,0.05550922
Baja California Sur,94156,0.04264530,0.005831633,0.04320773
Campeche,147824,0.03065538,0.009155606,0.03149669
Chiapas,920380,0.02906875,0.057004521,0.02860338
Chihuahua,355881,0.03493183,0.022041793,0.03515591
Ciudad de México,825595,0.04883833,0.051133931,0.04701864
Coahuila de Zaragoza,289557,0.04842732,0.017933960,0.04827614
Colima,60322,0.05078451,0.003736095,0.05127663


In [ ]:
# ------------------------------------------------ 3. Total de viviendas particulares habitadas en rezago habitacional y autoproducidas
tot_vivi_rezago <- dm %>%
  filter(vivi_autoprod == "Sí") %>%
  group_by(rezago) %>%
  summarise(
  viviendas=survey_total(vartype = "cv"),
  prop_viviendas = survey_prop(vartype = "cv"))%>%
  janitor::adorn_totals()

tot_vivi_rezago

rezago,viviendas,viviendas_cv,prop_viviendas,prop_viviendas_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
En rezago,5310725,0.01417899,0.3289243,0.012742457
Fuera de rezago,10835012,0.01111819,0.6710757,0.006245649
Total,16145737,0.02529718,1.0000000,0.018988106


In [ ]:
# ------------------------------------------------ 4. Total de residentes de viviendas particulares habitadas autoproducidas
tot_resid_autoprod <- dm %>%
  summarise(residentes = survey_total(tot_resid_autoprod, vartype = "cv", level = 0.95)) %>%
  janitor::adorn_totals()

tot_resid_autoprod

residentes,residentes_cv
<chr>,<dbl>
60426775,0.009749827
Total,0.009749827


In [ ]:
# ------------------------------------------------ 5. Total de residentes de viviendas particulares habitadas, autoproducidas y en rezago habitacional
tot_resid_autoprod_rezago <- dm %>%
  filter(rezago == "En rezago") %>%
  summarise(residentes = survey_total(tot_resid_autoprod, vartype = "cv", level = 0.95)) %>%
  janitor::adorn_totals()

tot_resid_autoprod_rezago

residentes,residentes_cv
<chr>,<dbl>
20456722,0.01655757
Total,0.01655757


In [ ]:
# ------------------------------------------------ 6. Porcentaje de viviendas particulares habitadas autoproducidas por estrato socioeconómico
pct_vivi_autoprod_est <- dm%>%
filter(vivi_autoprod == "Sí" )%>%
  #Estimación puntual
  group_by(est_socio,vivi_autoprod)%>%
  summarise(viviendas=survey_total(vartype="cv"))%>%
  ungroup()%>%
  #Seleccionar columnas est_socio y viviendas
  select(est_socio,viviendas)%>%
    #Calcula porcentaje
    mutate(pct=round(viviendas/sum(viviendas)*100,1),
    est_socio=case_when(est_socio==1 ~ "Bajo",
                        est_socio==2 ~ "Medio bajo",
                        est_socio==3 ~ "Medio alto",
                        est_socio==4 ~ "Alto")

    )%>%
    janitor::adorn_totals("row")

pct_vivi_autoprod_est

est_socio,viviendas,pct
<chr>,<dbl>,<dbl>
Bajo,5070986,31.4
Medio bajo,9008133,55.8
Medio alto,1616700,10.0
Alto,449918,2.8
Total,16145737,100.0


In [ ]:
# ------------------------------------------------ 7. Porcentaje de viviendas particulares habitadas autoproducidas por tamaño de localidad
pct_vivi_autoprod_loc <- dm %>%
  filter(vivi_autoprod == "Sí" )%>%
  group_by(tam_loc) %>%
  mutate(tam_loc = case_when(tam_loc == 4 ~ "Rural", TRUE ~ "Urbano")) %>%
  summarise(
    viviendas = survey_total(vartype = "cv"),
    pct = survey_prop(vartype = "cv")
  ) %>%
  ungroup() %>%
  janitor::adorn_totals()

pct_vivi_autoprod_loc

tam_loc,viviendas,viviendas_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Rural,5965595,0.009505942,0.3694842,0.009648745
Urbano,10180142,0.011991514,0.6305158,0.005654195
Total,16145737,0.021497456,1.0000000,0.015302940


In [ ]:
# ------------------------------------------------ 8. Porcentaje de viviendas particulares habitadas autoproducidas respecto del total de viviendas propias y en proceso de pago.
pct_vivi_pagos <- dm %>%
  filter(vivi_autoprod == "Sí" & (tenencia %in% c(3, 4))) %>%
  group_by(tenencia) %>%
  summarise(viviendas = survey_total(vartype = "cv"),
            pct = survey_prop(vartype = "cv"))%>%
                mutate(
    tenencia = case_when(
      tenencia == 3 ~ "Es propia pero la están pagando",
      tenencia == 4 ~ "	Es propia",
      TRUE ~ "No especificado"
    )
  ) %>%
    janitor::adorn_totals()

  pct_vivi_pagos

tenencia,viviendas,viviendas_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Es propia pero la están pagando,175956,0.060802190,0.01089799,0.0606072843
Es propia,15969781,0.008393368,0.98910201,0.0006677747
Total,16145737,0.069195558,1.00000000,0.0612750589


In [ ]:
# ------------------------------------------------ 9. Porcentaje de viviendas particulares habitadas autoproducidas con carencia por servicios básicos en la vivienda
pct_vivi_serv <- dm %>%
    filter(vivi_autoprod == "Sí") %>%
    group_by(ic_sbv) %>%
    summarise(
    viviendas = survey_total(vartype = "cv", level = 0.95),
    porcentaje = survey_prop(vartype = "cv", level = 0.95)) %>%
    mutate(
    ic_sbv = case_when(
      ic_sbv == 0 ~ "No presenta carencia",
      ic_sbv == 1 ~ "Presenta carencia",
      TRUE ~ "No especificado"
    ))%>%
  janitor::adorn_totals()

pct_vivi_serv

ic_sbv,viviendas,viviendas_cv,porcentaje,porcentaje_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
No presenta carencia,9597245,0.01223492,0.5944136,0.008053477
Presenta carencia,3424251,0.02040649,0.2120839,0.019378173
No especificado,3124241,0.01784330,0.1935025,0.016268615
Total,16145737,0.05048472,1.0000000,0.043700265


In [ ]:
# ------------------------------------------------ 10. Total de viviendas particulares habitadas autoproducidas en condición de hacinamiento
tot_vivi_hac <-dm %>%
    filter(vivi_autoprod == "Sí") %>%
    group_by(hacinamiento) %>%
    summarise(
    viviendas = survey_total(vartype = "cv", level = 0.95)
    ) %>%
    mutate(
    hacinamiento = case_when(
      hacinamiento == 0 ~ "No presenta condición de hacinamiento",
      hacinamiento == 1 ~ "Presenta condición de hacinamiento",
      TRUE ~ "No especificado"
    )
  ) %>%
  janitor::adorn_totals()
tot_vivi_hac

hacinamiento,viviendas,viviendas_cv
<chr>,<dbl>,<dbl>
No presenta condición de hacinamiento,15493279,0.008510755
Presenta condición de hacinamiento,652458,0.038639864
Total,16145737,0.047150619


In [ ]:
# ------------------------------------------------ 11. Porcentaje de viviendas particulares habitadas autoproducidas por antigüedad de la vivienda
tot_vivi_ant <- dm %>%
filter(vivi_autoprod == "Sí")%>%
#Crear variables de antigüedad
  mutate(catego=case_when(antiguedad==0 & antiguedad<5 ~ "0 y menos de 5 años",
                          antiguedad>=5 & antiguedad<10 ~ "5 y menos de 10 años",
                          antiguedad>=10 & antiguedad<20 ~ "10 y menos de 20 años",
                          antiguedad>=20 & antiguedad<30 ~ "20 y menos de 30 años",
                          antiguedad>=30 & antiguedad<40 ~ "30 y menos de 40 años",
                          antiguedad>=40 & antiguedad<50 ~ "40 y menos de 50 años",
                          antiguedad>=50 & antiguedad<60 ~ "50 y menos de 60 años",
                          antiguedad>=60 & antiguedad<70 ~ "60 y menos de 70 años",
                          antiguedad>=70 & antiguedad<80 ~ "70 y menos de 80 años",
                          antiguedad>=80 & antiguedad<90 ~ "80 y menos de 90 años",
                          antiguedad>=90 ~ "Más de 90 años",
                          TRUE ~ "No especificado"))%>%
  #Crear un grupo para crear las categorias
  group_by(catego)%>%
  summarise(viviendas=survey_total(vartype = "cv"))%>%
  ungroup()%>%
  filter(catego!="No especificado")%>%
  #Calcular porcentajes
  mutate(pct=round(viviendas/sum(viviendas)*100),
  #Ordenar las categorías
  catego=fct_relevel(catego,"0 y menos de 5 años",
                     "5 y menos de 10 años",
                     "10 y menos de 20 años",
                     "20 y menos de 30 años",
                     "30 y menos de 40 años",
                     "40 y menos de 50 años",
                     "50 y menos de 60 años",
                     "60 y menos de 70 años",
                     "70 y menos de 80 años",
                     "80 y menos de 90 años",
                     "Más de 90 años"))%>%
                         janitor::adorn_totals()
                       tot_vivi_ant

catego,viviendas,viviendas_cv,pct
<fct>,<dbl>,<dbl>,<dbl>
0 y menos de 5 años,24643,0.16632373,0
10 y menos de 20 años,3987518,0.01541123,27
20 y menos de 30 años,3704049,0.01732275,25
30 y menos de 40 años,2560059,0.01906267,18
40 y menos de 50 años,1464979,0.02568718,10
5 y menos de 10 años,1772632,0.02375868,12
50 y menos de 60 años,662278,0.03924789,5
60 y menos de 70 años,227908,0.06057870,2
70 y menos de 80 años,84435,0.09048110,1


In [ ]:
# ------------------------------------------------ 12. Porcentaje de viviendas particulares habitadas autoproducidas con cocina dormitorio
 pct_vivi_cdorm <- dm %>%
    filter(vivi_autoprod == "Sí") %>%
    group_by(cocina_dor) %>%
    summarise(
    viviendas = survey_total(vartype = "cv", level = 0.95),
    pct = survey_prop(vartype = "cv")) %>%
    mutate(
    cocina_dor = case_when(
      cocina_dor == 1 ~ "Sí",
      cocina_dor == 2 ~ "No",
      TRUE ~ "No especificado"
    )
  ) %>%
  janitor::adorn_totals()

pct_vivi_cdorm

cocina_dor,viviendas,viviendas_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Sí,828716,0.032530711,0.05132723,0.031148147
No,14202722,0.008917013,0.87965771,0.002758253
No especificado,1114299,0.028383669,0.06901506,0.027939407
Total,16145737,0.069831393,1.00000000,0.061845807


In [ ]:
# ------------------------------------------------ 13. Porcentaje de viviendas particulares habitadas autoproducidas por tipo de financiamiento
 pct_vivi_tipofin <- dm %>%
    filter(vivi_autoprod == "Sí") %>%
    group_by(tipo_finan) %>%
    summarise(
    viviendas = survey_total(vartype = c("cv"), level = 0.95),
    pct = survey_prop(vartype = "cv")) %>%
    mutate(
    tipo_finan = case_when(
      tipo_finan == 1 ~ "Crédito el INFONAVIT, FOVISSSTE o FONHAPO",
      tipo_finan == 2 ~ "Crédito un banco, SOFOL o caja de ahorro",
      tipo_finan == 3 ~ "Le dió crédito otra institución",
      tipo_finan == 4 ~ "Le prestó un familiar, amigo o prestamista",
      tipo_finan == 5 ~ "Usó sus propios recursos",
      TRUE ~ "No especificado"
    )
  ) %>%
  janitor::adorn_totals()

 pct_vivi_tipofin


tipo_finan,viviendas,viviendas_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
No especificado,61838,0.115265136,0.003829989,0.115152848
"Crédito el INFONAVIT, FOVISSSTE o FONHAPO",391548,0.046082803,0.024250860,0.045586648
"Crédito un banco, SOFOL o caja de ahorro",606987,0.039105488,0.037594258,0.038174895
Le dió crédito otra institución,350906,0.052485066,0.021733663,0.051924738
"Le prestó un familiar, amigo o prestamista",637707,0.038438203,0.039496928,0.037798081
Usó sus propios recursos,14096751,0.008974372,0.873094303,0.003012876
Total,16145737,0.300351069,1.000000001,0.291650086


In [ ]:
# ------------------------------------------------ 14. Porcentaje de viviendas particulares habitadas autoproducidas por tenencia de escrituras
 pct_vivi_escrit <- dm %>%
  filter(vivi_autoprod == "Sí") %>%
  group_by(escrituras) %>%
  summarise(
  viviendas = survey_total(vartype = "cv", level = 0.95),
  pct = survey_prop(vartype = "cv")) %>%
  mutate(
    escrituras = case_when(
      escrituras == 1 ~ "A nombre del dueño",
      escrituras == 2 ~ "A nombre de otra persona",
      escrituras == 3 ~ "No tiene escrituras",
      escrituras == 4 ~ "	No sabe",
      TRUE ~ "No especificado"
    )
  ) %>%
  janitor::adorn_totals()

pct_vivi_escrit

escrituras,viviendas,viviendas_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
A nombre del dueño,10305410,0.01062871,0.63827436,0.007254642
A nombre de otra persona,1198244,0.03009221,0.07421426,0.029205300
No tiene escrituras,4472385,0.01937736,0.27700098,0.016711877
No sabe,169698,0.08053758,0.01051039,0.080280071
Total,16145737,0.14063586,1.00000000,0.133451890


In [ ]:
# ------------------------------------------------ 15. Porcentaje de viviendas particulares habitadas autoproducidas por número de cuartos.
pct_vivi_ncuartos <- dm %>%
    filter(vivi_autoprod=="Sí") %>%
    # Crear variables de número de cuartos
    mutate(catego = case_when(num_cuarto >= 1 & num_cuarto <= 2 ~ "1 a 2 cuartos",
                              num_cuarto > 2 & num_cuarto <=4 ~ "Mayor a 2 hasta 4 cuartos",
                              num_cuarto >4 & num_cuarto <= 6 ~ "Mayor a 4 hasta 6 cuartos",
                              num_cuarto >6 & num_cuarto <= 8 ~ "Mayor a 6 hasta 8 cuartos",
                              num_cuarto >8 & num_cuarto <= 10 ~ "Mayor a 8  hasta 10 cuartos",
                              num_cuarto > 10 ~ "Mayor a 10",
                              TRUE ~ "No especificado")) %>%
    group_by(catego) %>%
    summarise(
    viviendas = survey_total(vartype = "cv"),
    pct=survey_prop(vartype="cv")) %>%
    # Calcular porcentaje
    mutate(# Ordenar categorías
           catego = fct_relevel(catego, "1 a 2 cuartos",
                                "Mayor a 2 hasta 4 cuartos",
                                "Mayor a 4 hasta 6 cuartos",
                                "Mayor a 6 hasta 8 cuartos",
                                "Mayor a 8  hasta 10 cuartos",
                                "Mayor a 10")) %>%
    janitor::adorn_totals()

  pct_vivi_ncuartos


catego,viviendas,viviendas_cv,pct,pct_cv
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1 a 2 cuartos,2648633,0.01960075,0.164045345,0.018185884
Mayor a 10,26408,0.18840513,0.001635602,0.188331012
Mayor a 2 hasta 4 cuartos,8566654,0.01076852,0.530583026,0.007010052
Mayor a 4 hasta 6 cuartos,4040650,0.01609648,0.250261106,0.013439173
Mayor a 6 hasta 8 cuartos,725617,0.03765868,0.044941708,0.036167762
Mayor a 8 hasta 10 cuartos,137775,0.08548902,0.008533212,0.085049600
Total,16145737,0.35801858,1.000000000,0.348183483


In [ ]:
# ------------------------------------------------ 16. Porcentaje de viviendas particulares habitadas autoproducidas por sexo del jefe del hogar
pct_vivi_sjefe <- dm %>%
  filter(vivi_autoprod == "Sí") %>%
  group_by(sexo_jefe) %>%
  summarise(
  viviendas = survey_total(vartype = c("cv"), level = 0.95),
    pct=survey_prop(vartype="cv")) %>%
  mutate(
    sexo_jefe = case_when(
      sexo_jefe == 1 ~ "Hombre",
      sexo_jefe == 2 ~ "Mujer",
      TRUE ~ "No especificado"
    )
  ) %>%
  janitor::adorn_totals()

pct_vivi_sjefe

sexo_jefe,viviendas,viviendas_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Hombre,9124586,0.01146122,0.5651390,0.007016231
Mujer,3897040,0.01541227,0.2413665,0.013600609
No especificado,3124111,0.01784400,0.1934945,0.016269157
Total,16145737,0.04471749,1.0000000,0.036885996


In [ ]:
# ------------------------------------------------ 17. Total de hogares en viviendas particulares habitadas autoproducidas por decil de ingreso
tot_hog_idecil <- dm %>%
  mutate(decil_ingreso = ntile(ing_cor, 10)) %>%
  group_by(decil_ingreso) %>%
  summarise(
  hogares = survey_total(tot_hog, vartype = "cv", level = 0.95)) %>%
  janitor::adorn_totals()

tot_hog_idecil

decil_ingreso,hogares,hogares_cv
<chr>,<dbl>,<dbl>
1,2497603,0.019962561
2,2722542,0.019354318
3,2773985,0.017405682
4,2810996,0.017877800
5,2896341,0.018274966
6,2869695,0.017405821
7,2939945,0.017032484
8,2986544,0.017055312
9,2951730,0.018060456


In [ ]:
# ------------------------------------------------ 18. Porcentaje de clase de hogar en viviendas particulares habitadas autoproducidas
pct_chog_autoprod <- dm %>%
  group_by(clase_hog) %>%
  summarise(
  viviendas = survey_total(tot_hog, vartype = c("cv"), level = 0.95),
    pct=survey_prop(vartype="cv")*100) %>%
  mutate(
    clase_hog = case_when(
      clase_hog == 1 ~ "Unipersonal",
      clase_hog == 2 ~ "Nuclear",
      clase_hog == 3 ~ "Ampliado",
      clase_hog == 4 ~ "Compuesto",
      clase_hog == 5 ~ "Corresidente",
      TRUE ~ "No especificado"
    )
  ) %>%
  janitor::adorn_totals()
pct_chog_autoprod

clase_hog,viviendas,viviendas_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Unipersonal,3560747,0.015912462,9.4218647,1.5615178
Nuclear,17589436,0.006535279,46.7846854,0.5134456
Ampliado,7175994,0.010773528,19.0609337,1.0177954
Compuesto,158621,0.068668700,0.4261315,6.8761344
Corresidente,76433,0.099038358,0.2062059,9.9166610
No especificado,8998892,0.006337066,24.1001788,0.5621228
Total,37560123,0.207265393,100.0000000,20.4476769


In [ ]:
# ------------------------------------------------ 19. Total de personas en pobreza dentro de viviendas particulares habitadas autoproducidas
tot_pers_pobreza <- dm %>%
  group_by(pobreza) %>%
  summarise(personas = survey_total(tot_resid_autoprod, vartype = c("cv"), level = 0.95))%>%
  mutate(
    pobreza = case_when(
      pobreza == 0 ~ "No pobre",
      pobreza == 1 ~ "Pobre",
      TRUE ~ "No especificado"
    )
  ) %>%
  janitor::adorn_totals()

tot_pers_pobreza

pobreza,personas,personas_cv
<chr>,<dbl>,<dbl>
No pobre,27503217,0.01397825
Pobre,21368957,0.01722923
No especificado,11554601,0.02237701
Total,60426775,0.05358450


In [ ]:
# ------------------------------------------------ 20. Total de personas en pobreza extrema dentro de viviendas particulares habitadas autoproducidas
tot_pers_pobrezae <- dm %>%
  group_by(pobreza_e) %>%
  summarise(
  viviendas = survey_total(tot_resid_autoprod, vartype = c("cv"), level = 0.95)
  )%>%
  mutate(
    pobreza_e = case_when(
      pobreza_e == 0 ~ "No pobre extremo",
      pobreza_e == 1 ~ "Pobre extremo",
      TRUE ~ "No especificado"
    )
  ) %>%
  janitor::adorn_totals()

tot_pers_pobreza

pobreza,personas,personas_cv
<chr>,<dbl>,<dbl>
No pobre,27503217,0.01397825
Pobre,21368957,0.01722923
No especificado,11554601,0.02237701
Total,60426775,0.05358450


In [ ]:
# Crear una lista con los nombres de las hojas y los data frames correspondientes
indica_resu <- list(
  'tot_vivi' = tot_vivi,
  'tot_vivi_nom_ent' = tot_vivi_nom_ent,
  'tot_vivi_rezago' = tot_vivi_rezago,
  'tot_resid_autoprod' = tot_resid_autoprod,
  'tot_resid_autoprod_rezago' = tot_resid_autoprod_rezago,
  'pct_vivi_autoprod_est' = pct_vivi_autoprod_est,
  'pct_vivi_autoprod_loc' = pct_vivi_autoprod_loc,
  'pct_vivi_pagos' = pct_vivi_pagos,
  'pct_vivi_serv' = pct_vivi_serv,
  'tot_vivi_hac' = tot_vivi_hac,
  'tot_vivi_ant' = tot_vivi_ant,
  'pct_vivi_cdorm' = pct_vivi_cdorm,
  'pct_vivi_tipofin' = pct_vivi_tipofin,
  'pct_vivi_escrit' = pct_vivi_escrit,
  'pct_vivi_ncuartos' = pct_vivi_ncuartos,
  'pct_vivi_sjefe' = pct_vivi_sjefe,
  'tot_hog_idecil' = tot_hog_idecil,
  'pct_chog_autoprod' = pct_chog_autoprod,
  'tot_pers_pobreza' = tot_pers_pobreza,
  'tot_pers_pobrezae' = tot_pers_pobrezae
)

# Escribir cada data frame en una hoja separada en el mismo archivo Excel
openxlsx::write.xlsx(indica_resu, file = 'indica_autoprod.xlsx')


#### Nombres de cada página con resultado

**tot_vivi** - 1. Total de viviendas particulares habitadas autoproducidas

**tot_vivi_nom_ent** - 2. Total de viviendas particulares habitadas autoproducidas por entidad federativa

**tot_vivi_rezago** - 3. Total de viviendas particulares habitadas en rezago habitacional y autoproducidas

**tot_resid_autoprod** - 4. Total de residentes de viviendas particulares habitadas autoproducidas

**tot_resid_autoprod_rezago** - 5. Total de residentes de viviendas particulares habitadas, autoproducidas y en rezago habitacional

**pct_vivi_autoprod_est** -  6. Porcentaje de viviendas particulares habitadas autoproducidas por estrato socioeconómico

**pct_vivi_autoprod_loc** - 7. Porcentaje de viviendas particulares habitadas autoproducidas por tamaño de localidad

**pct_vivi_pagos**- 8. Porcentaje de viviendas particulares habitadas autoproducidas respecto del total de viviendas propias y en proceso de pago

**pct_vivi_serv** - 9. Porcentaje de viviendas particulares habitadas autoproducidas con carencia por servicios básicos en la vivienda

**tot_vivi_hac** - 10. Total de viviendas particulares habitadas autoproducidas en condición de hacinamiento

**tot_vivi_ant** - 11. Porcentaje de viviendas particulares habitadas autoproducidas por antigüedad de la vivienda

**pct_vivi_cdorm** - 12. Porcentaje de viviendas particulares habitadas autoproducidas con cocina dormitorio

**pct_vivi_tipofin** - 13. Porcentaje de viviendas particulares habitadas autoproducidas por tipo de financiamiento

**pct_vivi_escrit** - 14. Porcentaje de viviendas particulares habitadas autoproducidas por tenencia de escrituras

**pct_vivi_ncuartos**  - 15. Porcentaje de viviendas particulares habitadas autoproducidas por número de cuartos.

**pct_vivi_sjefe**- 16. Porcentaje de viviendas particulares habitadas autoproducidas por sexo del jefe del hogar

**tot_hog_idecil**- 17. Total de hogares en viviendas particulares habitadas autoproducidas por decil de ingreso

**pct_chog_autoprod** - 18. Porcentaje de clase de hogar en viviendas particulares habitadas autoproducidas

**tot_pers_pobreza** - 19. Total de personas en pobreza dentro de viviendas particulares habitadas autoproducidas

**tot_pers_pobrezae** - 20. Total de personas en pobreza extrema dentro de viviendas particulares habitadas autoproducidas















